In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import geopandas as gpd
import geoplot.crs as gcrs
import geoplot as gplt
import re
from geopy.distance import great_circle
from shapely.geometry import Polygon, LineString, Point, MultiPolygon
from shapely import wkt
import folium

In [2]:
# Load the cleaned datasets for analysis and visualization
df = pd.read_csv('data/df_combined_cleaned_20250902.csv')
df2 = pd.read_csv('data/df_combined2_cleaned_20250902.csv')

/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_1756/947270512.py:2: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/df_combined_cleaned_20250902.csv')


In [5]:
# Convert data type of 'source' column to string
df['source'] = df['source'].astype(str)
df[df['source'] == '311'].shape

(360135, 12)

In [8]:
# Convert df to geo data frame
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs=4326)
df2 = gpd.GeoDataFrame(df2, geometry=gpd.points_from_xy(df2.longitude, df2.latitude), crs=4326)

In [75]:
from tqdm import tqdm

# Create a 311 map centered at San Francisco
m2 = folium.Map(location=[37.7749, -122.4194], zoom_start=12, tiles='CartoDB positron')

# Add points to the map
for _, row in tqdm(df2[(df2['source']=='311') & (df2['ground_truth']==1)].iterrows(), 
total=df2[(df2['source']=='311') & (df2['ground_truth']==1)].shape[0]):
    folium.CircleMarker(
        location = [row['latitude'], row['longitude']],
        radius = .3,
        color = "#e14b31",
        fill = True,
        opacity = 0.3,
    ).add_to(m2)

for _, row in tqdm(df2[(df2['source']=='311') & (df2['ground_truth']==0)].iterrows(), 
total=df2[(df2['source']=='311') & (df2['ground_truth']==0)].shape[0]):
    folium.CircleMarker(
        location = [row['latitude'], row['longitude']],
        radius = .3,
        color = "#1984c5",
        fill = True,
        opacity = 0.3,
    ).add_to(m2)

# Add a legend to the map for the different feature types
legend_html = '''
<div style="position: fixed;
     bottom: 50px; left: 50px; width: 170px; height: 90px;
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white; padding: 10px;">
     <b></b><br>
     <i style="background:#e14b31; width:10px; height:10px; display:inline-block;"></i> True Positive<br>
     <i style="background:#1984c5; width:10px; height:10px; display:inline-block;"></i> True Negative<br>
</div>
'''
# Add the legend HTML to the map
m2.get_root().html.add_child(folium.Element(legend_html))

# Save the maps to HTML files
m2.save('311_ground_truth_map_20250906.html')

100%|██████████| 23264/23264 [00:02<00:00, 8660.93it/s]


In [103]:
from tqdm import tqdm

# Create a Mapillary map centered at San Francisco
m3 = folium.Map(location=[37.7749, -122.4194], zoom_start=12, tiles='CartoDB positron')

# Add points to the map
for _, row in tqdm(df2[(df2['source']=='Mapillary') & (df2['ground_truth']==0)].iterrows(), 
total=df2[(df2['source']=='Mapillary') & (df2['ground_truth']==0)].shape[0]):
    folium.CircleMarker(
        location = [row['latitude'], row['longitude']],
        radius = .3,
        color = "#1984c5",
        fill = True,
        opacity = 0.3,
    ).add_to(m3)

for _, row in tqdm(df2[(df2['source']=='Mapillary') & (df2['ground_truth']==1)].iterrows(), 
total=df2[(df2['source']=='Mapillary') & (df2['ground_truth']==1)].shape[0]):
    folium.CircleMarker(
        location = [row['latitude'], row['longitude']],
        radius = 1.5,
        color = "#e14b31",
        fill = True,
        opacity = 1,
    ).add_to(m3)

# Add a legend to the map for the different feature types
legend_html = '''
<div style="position: fixed;
     bottom: 50px; left: 50px; width: 170px; height: 85px;
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white; padding: 10px;">
     <b></b><br>
     <i style="background:#e14b31; width:10px; height:10px; display:inline-block;"></i> True Positive<br>
     <i style="background:#1984c5; width:10px; height:10px; display:inline-block;"></i> True Negative<br>
</div>
'''
# Add the legend HTML to the map
m3.get_root().html.add_child(folium.Element(legend_html))

# Save the Mapillary ground truth map to an HTML file
m3.save('mapillary_ground_truth_map_20250906.html')

100%|██████████| 208/208 [00:00<00:00, 961.98it/s]


In [104]:
from tqdm import tqdm

# Load the amenities data for San Francisco from OpenStreetMap
df_map = pd.read_csv('data/sf_osm_amenities_2016_2024.csv')

# Filter the dataframe to only include data from the snapshot date '2024-05-31'
df_map = df_map[df_map['snapshot_date']=='2024-05-31']

# Filter the dataframe to only include amenities of interest
amenities_of_interest = ['bench', 'restaurant', 'fast_food', 'place_of_worship', 'school', 'bank', 
                         'atm', 'social_facility', 'shelter', 'university', 'college', 'police', 'cinema']

df_map = df_map[df_map['amenity'].isin(amenities_of_interest)]

# Create a amenity map centered at San Francisco
m4 = folium.Map(location=[37.7749, -122.4194], zoom_start=12, tiles='CartoDB positron')

# Add amenity points to the map with different colors based on amenity type
# Iterate through each row in the dataframe and add a CircleMarker for each amenity type
for _, row in tqdm(df_map.iterrows(), total=df_map.shape[0]):
    color = ""  # Default color
    if row['amenity'] == 'restaurant':
        color = "#1984c5"
    elif row['amenity'] == 'bench':
        color = "#e14b31"
    elif row['amenity'] == 'fast_food':
        color = "#2ca02c"
    elif row['amenity'] == 'place_of_worship':
        color = "#ff7f0e"
    elif row['amenity'] == 'school':
        color = "#9467bd"
    elif row['amenity'] == 'bank':
        color = "#8c564b"
    elif row['amenity'] == 'atm':
        color = "#EBCB3C"
    elif row['amenity'] == 'social_facility':
        color = "#17becf"
    elif row['amenity'] == 'shelter':
        color = "#bcbd22"
    elif row['amenity'] == 'university':
        color = "#ffbb78"
    elif row['amenity'] == 'college':
        color = "#aec7e8"
    elif row['amenity'] == 'police':
        color = "#c49c94"
    elif row['amenity'] == 'cinema':
        color = "#98df8a"
    folium.CircleMarker(
        location = [row['lat'], row['lon']],
        radius = 1.5,
        color = color,
        fill = True,
        opacity = 0.5,
    ).add_to(m4)

# Add a legend to the map for the different amenity types
legend_html = '''
<div style="position: fixed;
     bottom: 50px; left: 50px; width: 170px; height: 300px;
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white; padding: 10px;">
     <b>Amenity Types</b><br>
     <i style="background:#1984c5; width:10px; height:10px; display:inline-block;"></i> Restaurant<br>
     <i style="background:#e14b31; width:10px; height:10px; display:inline-block;"></i> Bench<br>
     <i style="background:#2ca02c; width:10px; height:10px; display:inline-block;"></i> Fast Food<br>
     <i style="background:#ff7f0e; width:10px; height:10px; display:inline-block;"></i> Place of Worship<br>
     <i style="background:#9467bd; width:10px; height:10px; display:inline-block;"></i> School<br>
     <i style="background:#8c564b; width:10px; height:10px; display:inline-block;"></i> Bank<br>
     <i style="background:#d62728; width:10px; height:10px; display:inline-block;"></i> ATM<br>
     <i style="background:#17becf; width:10px; height:10px; display:inline-block;"></i> Social Facility<br>
     <i style="background:#bcbd22; width:10px; height:10px; display:inline-block;"></i> Shelter<br>
     <i style="background:#ffbb78; width:10px; height:10px; display:inline-block;"></i> University<br>
     <i style="background:#aec7e8; width:10px; height:10px; display:inline-block;"></i> College<br>
     <i style="background:#c49c94; width:10px; height:10px; display:inline-block;"></i> Police<br>
     <i style="background:#98df8a; width:10px; height:10px; display:inline-block;"></i> Cinema<br>
</div>
'''

# Add the legend HTML to the map with the updated title "Amenity Types"
m4.get_root().html.add_child(folium.Element(legend_html))

# Save the amenity map to an HTML file
m4.save('amenity_map_20250906.html')


100%|██████████| 6584/6584 [00:00<00:00, 6837.97it/s]


In [102]:
from tqdm import tqdm
import folium.plugins as plugins

# Load the amenities data for San Francisco from OpenStreetMap
df_map = pd.read_csv('data/sf_osm_amenities_2016_2024.csv')

# Filter the dataframe to only include data from the snapshot date '2024-05-31'
df_map = df_map[df_map['snapshot_date']=='2024-05-31']

# Filter the dataframe to only include "bridge" and "highway_link"
df_map = df_map[df_map['feature_type'].isin(['bridge', 'highway_link'])]

# Create a amenity map centered at San Francisco
m5 = folium.Map(location=[37.7749, -122.4194], zoom_start=12, tiles='CartoDB positron')

# Add structure (bridge and highway_link) points to the map with different colors and shapes based on feature type
for _, row in tqdm(df_map.iterrows(), total=df_map.shape[0]):
    color = ""  # Default color
    if row['feature_type'] == 'bridge':
        color = "#1984c5"
        icon = "bridge"
    elif row['feature_type'] == 'highway_link':
        color = "#e14b31"
        icon = "road"
    folium.Marker(
        location=[row['lat'], row['lon']],
        icon=plugins.BeautifyIcon(color=color, icon=icon, icon_shape="circle",
                                  border_color=color),
        opacity = 0.8,
    ).add_to(m5)

# Add a legend to the map for the different feature types
legend_html = '''
<div style="position: fixed;
     bottom: 50px; left: 50px; width: 170px; height: 100px;
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white; padding: 10px;">
     <b>Structure Types</b><br>
     <i style="background:#1984c5; width:10px; height:10px; display:inline-block;"></i> &nbsp;Bridge<br>
     <i style="background:#e14b31; width:10px; height:10px; display:inline-block;"></i> &nbsp;Highway Ramp<br>
</div>
'''

# Add the legend HTML to the map with the updated title "Feature Types"
m5.get_root().html.add_child(folium.Element(legend_html))

# Save the structure map to an HTML file
m5.save('structure_map_20250906.html')


100%|██████████| 1191/1191 [00:00<00:00, 8720.65it/s]


In [99]:
df_temp = pd.DataFrame({
    'feature_type': ['highway_link', 'bridge'],
    'lat': [37.964216, 37.947189], 
    'lon': [-122.759320, -122.738724],
})

In [100]:
df_temp

,feature_type,lat,lon
0,highway_link,37.964216,-122.759320
1,bridge,37.947189,-122.738724


In [101]:
from tqdm import tqdm
import folium.plugins as plugins

# Create a amenity map centered at San Francisco
m6 = folium.Map(location=[37.970954, -122.702310], zoom_start=12, tiles='cartodbpositron')

# Add structure (bridge and highway_link) points to the map with different colors and shapes based on feature type
for _, row in tqdm(df_temp.iterrows(), total=df_temp.shape[0]):
    color = ""  # Default color
    if row['feature_type'] == 'bridge':
        color = "#1984c5"
        icon = "bridge"
    elif row['feature_type'] == 'highway_link':
        color = "#e14b31"
        icon = "road"
    folium.Marker(
        location=[row['lat'], row['lon']],
        icon=plugins.BeautifyIcon(color=color, icon=icon, icon_shape="circle",
                                  border_color=color, size="large"),
        opacity = 1,
    ).add_to(m6)

# Add a legend to the map for the different feature types
legend_html = '''
<div style="position: fixed;
     bottom: 50px; left: 50px; width: 170px; height: 100px;
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white; padding: 10px;">
     <b>Structure Types</b><br>
     <i style="background:#1984c5; width:10px; height:10px; display:inline-block;"></i> &nbsp;Bridge<br>
     <i style="background:#e14b31; width:10px; height:10px; display:inline-block;"></i> &nbsp;Highway Link<br>
</div>
'''

# Add the legend HTML to the map with the updated title "Feature Types"
m6.get_root().html.add_child(folium.Element(legend_html))

# Save the structure map to an HTML file
m6.save('structure_map_test_20250906.html')


100%|██████████| 2/2 [00:00<00:00, 1243.13it/s]
